In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import geopandas as gpd
from shapely.geometry import Point

pd.set_option('display.max_columns', None)

## Data Acquisition

In [ ]:
data = "Crime_Data_from_2020_to_Present_20251006.csv"
districts = gpd.read_file("LA_City_Council_Districts_(Adopted_2021).geojson")
df = pd.read_csv(data, sep=',', encoding='latin-1')
df.head(2)

## Data Wrangling

### Deduplikasi

In [ ]:
print("\n Jumlah nilai hilang per kolom:")
print(df.shape)
print(df.isna().sum()[lambda x: x > 0])

In [ ]:
df[['Mocodes', 'Vict Sex', 'Vict Descent', 'Premis Cd',
    'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status',
    'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'Cross Street', 'DISTRICT']].nunique()

### Merging

In [ ]:
df['geometry'] = df.apply(lambda row: Point(row['LON'], row['LAT']), axis=1)
dist_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=districts.crs)
joined = gpd.sjoin(dist_gdf, districts[['District', 'geometry']], how='left', predicate='within')

df['DISTRICT'] = joined['District']
df = df.drop(columns='geometry')

### Operasi aritmatika dengan NaN

In [ ]:
dfvict = df.copy()
dfvict['VictAge_x_freq'] = df['Vict Age'] * (df['DISTRICT'] / df['DISTRICT'].max())

Operasi umur korban dikali besar distrik untuk melihat di daerah mana korban yang lebih tua atau muda lebih banyak. Karena kolom DISTRICT mengandung nilai NaN, hasil perkalian pada baris-baris tersebut juga akan menghasilkan NaN. Sehingga nilai NaN tersebut harus diatasi terlebih dahulu.

### Penanganan nilai hilang

#### District & Cross Street

In [ ]:
print("Baris dengan DISTRICT kosong:")
df[df['DISTRICT'].isna()][['LOCATION', 'LAT', 'LON', 'DISTRICT']].head(3)

Setelah dilakukan pemeriksaan, diketahui bahwa laporan-laporan kejahatan dengan nilai NaN pada kolom **District** terjadi di luar wilayah administratif Kota Los Angeles. Oleh karena itu, data dengan nilai NaN pada kolom tersebut dihapus agar analisis difokuskan pada kejadian yang benar-benar terjadi di dalam wilayah Kota LA.

Sementara itu, kolom **Cross Street** memiliki jumlah data kosong (missing values) yang terlalu besar dibandingkan keseluruhan data. Karena minimnya informasi yang dapat diperoleh dari kolom ini, serta potensi rendahnya kontribusi terhadap analisis, maka kolom tersebut dihapus untuk menyederhanakan dataset.

In [ ]:
before = len(df)
print(f"\n Jumlah baris sebelum: {before}")

df = df.dropna(subset=['DISTRICT'])

print(f" Jumlah baris sesudah: {len(df)}")
print(f" Baris dihapus (karena DISTRICT NaN): {before - len(df)}")

#### Weapon Used CD & Desc

In [ ]:
print(f'Jumlah senjata: {len(df['Weapon Desc'].unique())}')
print(f'Nilai unik senjata: {df['Weapon Desc'].unique()}')

In [ ]:
df[df['Weapon Used Cd'].isna()][['Crm Cd Desc']].drop_duplicates(subset='Crm Cd Desc').head()

Berdasarkan deskripsi kasus kriminal dengan nilai Weapon yang bernilai NaN, dapat diasumsikan bahwa tindak kejahatan tersebut dilakukan tanpa menggunakan senjata. Oleh karena itu, nilai pada kolom Weapon Description dapat diisi dengan kategori ‘UNKNOWN WEAPON/OTHER WEAPON’

In [ ]:
df['Weapon Desc'] = df['Weapon Desc'].fillna('UNKNOWN WEAPON/OTHER WEAPON')

print(df['Weapon Desc'].value_counts().head())
print("\nJumlah missing setelah imputasi:", df['Weapon Desc'].isna().sum())

#### Status

In [ ]:
print("\nJumlah missing status desc:", df['Status Desc'].isna().sum())

Karena kolom Status merupakan representasi kode dari kolom Status Description, dan seluruh data pada Status Description telah terisi lengkap tanpa nilai yang hilang, maka kolom Status dapat dihapus dari dataset

#### Crm Cd 1 - 4

In [ ]:
df[df['Crm Cd 1'].isna()]

Karena tidak terdapat informasi yang menjelaskan makna dari setiap nilai pada kolom Crm Cd 1 hingga Crm Cd 4, maka kolom-kolom tersebut dihapus dari dataset.

#### Premis Cd & Desc

In [ ]:
print(f'Jumlah Missing Premis Cd: {df['Premis Cd'].isna().sum()}')
print(f'Jumlah Missing Premis Desc: {df['Premis Desc'].isna().sum()}')

print(f'\nJumlah Nilai Unik Premis Cd: {df['Premis Cd'].nunique()}')
print(f'Jumlah Nilai Unik Premis Desc: {df['Premis Desc'].nunique()}')

In [ ]:
df.groupby('Premis Cd')['Premis Desc'].nunique().sort_values(ascending=False)

Hal ini menunjukkan bahwa terdapat beberapa deskripsi Premis yang tidak lengkap, yaitu pada data dengan Premis Cd yang nilai Premis Desc-nya kosong. Setelah dilakukan penelusuran melalui internet, nilai Premis Cd yang hilang tersebut tidak ditemukan padanan deskripsinya. Karena data tersebut tidak memberikan informasi yang bermakna, maka baris tersebut dihapus dari dataset.

In [ ]:
before = len(df)
df = df[~((df['Premis Cd'].notna()) & (df['Premis Desc'].isna()))]
after = len(df)

print(f"\n Jumlah baris sebelum: {before}")
print(f" Jumlah baris sesudah: {after}")
print(f" Baris dihapus: {before-after}")

In [ ]:
print(f'Jumlah Missing Premis Desc: {df['Premis Desc'].isna().sum()}')

Karena data tersebut tidak tersedia, dilakukan proses imputasi berbasis model dengan memanfaatkan variabel lain sebagai fitur untuk mengisi nilai yang hilang pada kolom Premis Desc.

In [ ]:
df[df['Premis Desc'].isna()].head(3)

#### Vict Sex & Descent

In [ ]:
# Vict Descent
print(f'Nilai unik ras korban: {df['Vict Descent'].unique()}')
print(f'Jumlah ras -: {len(df[df['Vict Descent'] == '-'])}')
print(f'Jumlah ras Unknown: {len(df[df['Vict Descent'] == 'X'])}')

# Vict Sex
print(f'\nNilai unik gender korban: {df['Vict Sex'].unique()}')

Karena data kriminalitas tersebut telah ditelusuri namun tidak ditemukan informasi pendukungnya, maka nilai pada kolom Vict Sex dan Vict Descent yang bernilai ‘-’ diubah menjadi ‘X’.

In [ ]:
df.loc[df['Vict Descent'] == '-', 'Vict Descent'] = 'X'
df.loc[df['Vict Sex'] == '-', 'Vict Sex'] = 'X'

In [ ]:
print(f"Jumlah Missing Vict Sex: {df['Vict Sex'].isna().sum()}")
print(f"Jumlah Missing Vict Descent: {df['Vict Descent'].isna().sum()}")

print(f"Jumlah Unknown Vict Sex: {(df['Vict Sex'] == 'X').sum()}")
print(f"Jumlah Unknown Vict Descent: {(df['Vict Descent'] == 'X').sum()}")

Karena jumlah data yang hilang masih cukup besar, maka akan dilakukan proses imputasi untuk mengisi nilai yang missing tersebut.

#### Imputasi ( Premis, Vict Sex, Descent )

In [ ]:
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], errors='coerce')
df['DATE_TIME_OCC'] = pd.to_datetime(
        df['DATE OCC'].dt.strftime('%Y-%m-%d') + ' ' +
        df['TIME OCC'].astype(str).str.zfill(4),
        format='%Y-%m-%d %H%M',
        errors='coerce'
    )
df.drop(columns=['DATE OCC', 'TIME OCC'], inplace=True, errors='ignore')

Dilakukan penyesuaian format tanggal pada kolom tersebut agar proses imputasi dapat berjalan dengan lancar.

In [ ]:
# Distribusi frekuensi untuk masing-masing kolom
vict_sex_dist = df['Vict Sex'].value_counts(dropna=False)
vict_descent_dist = df['Vict Descent'].value_counts(dropna=False)
premis_desc_dist = df['Premis Desc'].value_counts(dropna=False)

print("Distribusi Vict Sex:\n",vict_sex_dist)
print("\nDistribusi Vict Descent:\n", vict_descent_dist)
print("\nDistribusi Premis Desc:\n", premis_desc_dist)

In [ ]:
print(df.isna().sum()[lambda x: x > 0])

Karena beberapa kolom lain memiliki nilai kosong dan akan dihapus, maka kolom-kolom tersebut dikecualikan terlebih dahulu dari proses imputasi pada kolom Premis Desc, Vict Sex, dan Vict Descent.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np

exclude_cols = ['Mocodes', 'Premis Cd', 'Weapon Used Cd',
                'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4',
                'Cross Street', 'LAT', 'LON', 'Date Rptd', 'DATE_TIME_OCC']
exclude_cols = [c for c in exclude_cols if c in df.columns]

impute_cols = ['Vict Sex', 'Vict Descent', 'Premis Desc']
impute_cols = [c for c in impute_cols if c in df.columns]

# Hanya ambil kolom yang mau diimputasi dan prediktor
use_cols = [c for c in df.columns if c not in exclude_cols]
df_sub = df[use_cols].copy()

In [ ]:
for col in df_sub.columns:
    if df_sub[col].dtype == 'object':
        df_sub[col] = df_sub[col].astype('category')
    if str(df_sub[col].dtype) == 'category':
        df_sub[col] = df_sub[col].cat.codes.replace(-1, np.nan)

imputer = IterativeImputer(random_state=42, max_iter=10)
df_imputed = pd.DataFrame(imputer.fit_transform(df_sub), columns=df_sub.columns, index=df.index)

# bulatkan hasil kategorik ke kode terdekat lalu kembalikan ke kategori asli
for col in df_sub.columns:
    if df[col].dtype == 'object' or str(df[col].dtype) == 'category':
        df_imputed[col] = np.round(df_imputed[col]).astype(int)
        cats = df[col].astype('category').cat.categories
        df_imputed[col] = pd.Categorical.from_codes(
            df_imputed[col].clip(0, len(cats)-1), cats
        )

# update kolom yang diimputasi
for col in impute_cols:
    df[col] = df_imputed[col]

print(df[impute_cols].isna().sum())

In [ ]:
print("\nHasil imputasi (5 baris pertama):")
print(df[impute_cols].head())

print("\nJumlah missing value setelah imputasi:")
print(df[impute_cols].isna().sum())

### Rename axis/index

In [ ]:
df = df.drop(columns=[
    'AREA', 'DR_NO', 'Crm Cd', 'Mocodes', 'Premis Cd',
    'Weapon Used Cd', 'Status', 'Crm Cd 1', 'Crm Cd 2',
    'Crm Cd 3', 'Crm Cd 4'
])

df.rename(columns={
    'Crm Cd Desc': 'Crm',
    'Premis Desc': 'Premis',
    'Weapon Desc': 'Weapon',
    'Status Desc': 'Status'
}, inplace=True)

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
print("\n🧾 Nama kolom setelah transformasi:")
print(df.columns.tolist())

### Diskritisasi / Binning


In [ ]:
df['vict_age_bin'] = pd.qcut(df['vict_age'], q=4, labels=['Muda', 'Dewasa', 'Paruh Baya', 'Tua'])

### Deteksi Outlier & filtering

#### Numerik

In [ ]:
import statsmodels.api as sm

sample = df['vict_age'].sample(min(10000, len(df['vict_age'].dropna())), random_state=1)

# 🔹 Histogram + KDE
plt.figure(figsize=(7,4))
sns.histplot(sample, kde=True, bins=30)
plt.title(f'Distribusi {'vict_age'} (Sample)')
plt.xlabel('vict_age')
plt.ylabel('Frekuensi')
plt.show()

# 🔹 Q-Q Plot
plt.figure(figsize=(5,5))
sm.qqplot(sample, line='s')
plt.title(f'Q-Q Plot {'vict_age'} (Sample)')
plt.show()

In [ ]:
# 🔹 Uji normalitas menggunakan D’Agostino-Pearson karena data leih dari 5000
print("\n=== Uji Normalitas ===")
print("D’Agostino-Pearson:", stats.normaltest(df['vict_age']))

Nilai p-value-nya 0.0 (sangat kecil), artinya data tidak berdistribusi normal menurut tes D’Agostino-Pearson. Sehingga dilakukan pendeteksian outlier menggunakan IQR pada kolom umur

In [ ]:
Q1 = df['vict_age'].quantile(0.25)
Q3 = df['vict_age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Kebijakan Filtering
outliers = df[(df['vict_age'] < lower_bound) | (df['vict_age'] > upper_bound)]

print(f"Jumlah outlier: {len(outliers)}")
print(outliers[['vict_age']])

Karena data tersebut secara format dan konteks tampak masuk akal sebagai data kejadian kriminal, maka baris tersebut akan dibiarkan

#### Kategorik

In [ ]:
df['district'] = df['district'].astype('object')
df['rpt_dist_no'] = df['rpt_dist_no'].astype('object')
df['part_1-2'] = df['part_1-2'].astype('object')
df['vict_age_bin'] = df['vict_age_bin'].astype('object')

df.info()

In [ ]:
cat_cols = df.select_dtypes(include='object').columns.difference(['date_rptd', 'date_time_occ'])

for col in cat_cols:
    freq = df[col].value_counts(normalize=True)
    rare_values = freq[freq < 0.01]  # contoh threshold <1% dianggap langka
    print(f"\n\033[1m\033[92mKolom: {col}\033[0m")
    print(f"Nilai langka (<1% frekuensi):")
    print(rare_values)

In [ ]:
df[df['vict_age'] == 120]

Kategori langka pada data ini tetap dipertahankan karena mereka mengandung informasi penting dan detail spesifik, seperti jenis kejahatan yang jarang terjadi. Menghapus atau menggabungkan kategori-kategori ini bisa menghilangkan insight berharga yang berguna untuk analisis mendalam. Oleh karena itu, kategori langka dibiarkan utuh agar detail tersebut tetap terjaga.

In [ ]:
df.to_csv("Crime_Data_Clean.csv", index=False)